In [ ]:

## 01 Data preprocessing & Normalization & Harmonization
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import scanpy as sc
import anndata as ad
import harmonypy 
import scanpy.external as sce
main_path="/path/to/workdir/"
saw_output="/path/to/saw_output/"
os.chdir(saw_output)
samples=os.listdir()

data_paths=[i+'/outs/analysis/'+i+'.cellbin_1.0.adjusted.h5ad' for i in samples]

for i,j in enumerate(data_path):
    tmp=sc.read_h5ad(j)
    tmp.obs['slide_id']=samples[i]
    tmp.obs['slide']=sample[i]
    tmp.var['EnsID']=tmp.var.index
    tmp.var.real_gene_name=tmp.var.real_gene_name.astype('str')
    tmp.var.index=tmp.var.real_gene_name
    tmp.var_names_make_unique("-")
    tmp.obs['CID']=tmp.obs.index
    adata_list.append(tmp)


adata=ad.concat(adata_list,label="slide_id")

sc.pl.highest_expr_genes(adata, n_top=20, )
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
adata.obs.index=adata.obs.index.str.cat(adata.obs.batch,sep="-")

adata=adata[(adata.obs.n_genes_by_counts < 10000)& (adata.obs.n_genes_by_counts > 50)&(adata.obs.total_counts > 100)
    &(adata.obs.total_counts < 8000) &adata.obs.pct_counts_mt.lt(20), :]

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, batch_key='slide',flavor='seurat_v3_paper',n_top_genes=3000)

adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata,max_value=10)
sc.tl.pca(adata, use_highly_variable=True,svd_solver='arpack')
sce.pp.harmony_integrate(adata,key='slide')
sc.pp.neighbors(adata,use_rep='X_pca_harmony',n_pcs=30)
sc.tl.umap(adata)
sc.tl.louvain(adata,resolution=4.0)
sc.pl.umap(adata, color=['slide'], use_raw=True)
adata.raw.to_adata().write("/path/to/integrated_st.h5ad")

## plotting marker gene expression

markers=['TP63','LGR5','HOPX','KISS1','HLA-G','KRT7','LAMA2','COL3A1','ACTA2','DLK1','CD34',
         'PECAM1','MMP1','VWF','FOXO1','IGFBP1','PAEP','PAX8','GNLY','CD3E','MRC1']

sc.pl.dotplot(adata, markers,categories_order=['VCT','SCT','EVT','FB','PV','fVEC','mVEC','DSC','Epi','immune','HB'],dot_max=0.8,
              groupby='subclass',use_raw=True,standard_scale='var')

In [ ]:

## 02. Adjusting spatial coordianate
import numpy as np
offset_x=adata.obs.x.max()
offset_y=adata.obs.y.max()
samples=adata.obs['slide'].unique()
adata.obsm['spatial']=np.asarray(adata.obs[['x','y']])
adata.obsm['shift_spatial']=adata.obsm['spatial']
ncols=6
for i, sample in enumerate(samples):
    sample_idx = adata.obs["slide"] == sample
    if i<ncols:
        adata.obsm["shift_spatial"][sample_idx,0] += i * offset_x
    else:
        adata.obsm["shift_spatial"][sample_idx,0] += (i-ncols) * offset_x
        adata.obsm["shift_spatial"][sample_idx,1] += 1 * offset_y
sc.pl.embedding(adata,basis="shift_spatial")

In [ ]:
## 03 Cell community analysis


import stereo as st
from stereo.core.stereo_exp_data import AnnBasedStereoExpData
from stereo.algorithm.community_detection import CommunityDetection
from stereo.core.ms_data import MSData

data=st.io.read_h5ad("/path/to/integrated_st.h5ad")
ccd = data.tl.community_detection(
            annotation='subclass',
            out_path='./community/',
            win_sizes='300',
            sliding_steps='10',
            scatter_thres=0.12,
            downsample_rate=80,
            cluster_algo='agglomerative',
            n_clusters=12,
            resolution=0.25,
            plotting=3,num_threads=20,
            hide_plots=False,
            min_count_per_type =0.01)



## Plotting cell type compositiona across cell community
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype']=42
composition = adata.obs.groupby(['ccd', 'subclass']).size().reset_index(name='count')
composition['proportion'] = composition.groupby('ccd')['count'].transform(lambda x: x / x.sum())
plt.figure(figsize=(10, 5))
composition_pivot = composition.pivot(index="ccd", columns="subclass", values="proportion").fillna(0)
composition_pivot.plot(kind="bar", stacked=True, colormap="tab20", figsize=(10, 5))
plt.title("Cell Composition by Distance (Stacked Bar Chart)")
plt.xticks(rotation=45)

In [ ]:
## 04.Celltype co-occurence/neighborhood enrichment
import squidpy as sq
from scipy.cluster.hierarchy import linkage

sq.gr.spatial_neighbors(adata spatial_key='shift_spatial', coord_type="generic",delaunay=False)
adata2.obs['subclass']=pd.Categorical(adata2.obs['subclass'])
sq.gr.nhood_enrichment(adata, cluster_key="subclass",n_perms=1000)
df=pd.DataFrame(adata2.uns['subclass_nhood_enrichment']['count'])
df.index=adata.obs.subclass.cat.categories
df.columns=adata.obs.subclass.cat.categories
df=(df.div(df.sum(1),0)+df.div(df.sum(0),1))/2
linkage_matrix = linkage(df, method='ward')
sns.clustermap(df, cmap='viridis',col_linkage=linkage_matrix,row_linkage=linkage_matrix)

In [ ]:
## 05 Spatial Distance Analysis

## calculate the signed distance to the interface
from shapely.ops import nearest_points, LineString
from shapely.geometry import Point, Polygon
import geopandas as gpd

def signed_distance_interface(point, line):
    nearest_point_on_line=nearest_points(line, point)[0]
    distance=point.distance(nearest_point_on_line)
    coords = line.coords
    sign_distance = 0
    seg = None
    #normal = None
    for i in range(len(coords) - 1):
        seg_start, seg_end = Point(coords[i]), Point(coords[i+1])
        segment = LineString([seg_start, seg_end])
        if point.distance(segment)==distance:
            tangent = (seg_end.x - seg_start.x, seg_end.y - seg_start.y)
        # Compute normal vector (perpendicular to tangent)
            #normal = (tangent[0], tangent[1])
            seg = (seg_start, seg_end)
    if seg:
            #point_vec = (point.x - nearest_point_on_line.x, point.y - nearest_point_on_line.y)
        point_vec = (point.x - seg[0].x, point.y - seg[0].y)
        cross = tangent[0] * point_vec[1] - tangent[1] * point_vec[0]
        #sign = 1 if (point_vec[0] * normal[0] + point_vec[1] * normal[1]) > 0 else -1
        sign = 1 if cross >0 else -1
        sign_distance = distance * sign

    return sign_distance
    
## signed distance to BV
def signed_distance_bv(point, polygons):
    if polygons.contains(point).any():
        polygon=polygons[polygons.contains(point)]
        distance=polygon.geometry.boundary.distance(point).min()*-1
    # Compute distance to nearest polygon
    else:
        distance = polygons.geometry.boundary.distance(point).min()
    # If the point is inside any polygon, return negative distance
    return distance

## compute signed distance on masked sample

sc.set_figure_params(dpi=80)
ids=os.listdir('/path/to/saw_output/')
df=[]
for id in ids:
    mask_gdf = gpd.read_file(os.path.join("/path/to/tissue_mask/",id+"_seg.geojson")) ## load coordinates of sample mask and interface lines.
    interface_gdf = gpd.read_file(os.path.join("/path/to/tissue_mask/",id+"_interface.geojson"))
    adata2=adata[adata.obs.slide.isin([id]),:]
    points=[Point(i[0],i[1]) for i in adata2.obs[['x','y']].values]
    if mask_gdf.shape[0]==2:
        sample_ids=[0,1]
    else:
        sample_ids=[0]
    for sample in sample_ids
    ## selecting cells masked by each individual samples
        points_df=gpd.GeoDataFrame({'geometry': points})
        points_df.index=adata2.obs.index
        points_df=gpd.sjoin(points_df,mask_gdf.iloc[[sample]])
        adata2=adata2[points_df.index,:]

        signed_distances,whichline = [],[]
        # iterative on all cells to track the o track the minimum signed distance and corresponding line
        for j, point in enumerate(points_df.geometry):
            min_signed_distance = None
            nearest_line = sample  
            s_distance = signed_distance_interface(point, interface_gdf.geometry[sample])
            signed_distances.append(s_distance)
            whichline.append(nearest_line)
        points_df['signed_distance'] = signed_distances
        points_df['nearest_line'] = whichline
        df.append(points_df)
df=pd.concat(df,axis=0)
df['celltype']=adata.obs.loc[df.index,'celltype']
df['subclass']=adata.obs.loc[df.index,'subclass']
df.to_csv("signed_distance_interface.tsv",sep="\t",index=True)

## compute signed distance on masked sample
for id in ids:
    mask_gdf = gpd.read_file(os.path.join("/path/to/tissue_mask/",id+"_seg.geojson")) ## load coordinates of sample mask and bv shapes.
    bv_gdf = gpd.read_file(os.path.join("/path/to/tissue_mask/",id+"_BV.geojson"))
    bv_gdf=bv_gdf.drop(bvs.shape[0]-1)
    adata2=adata[adata.obs.slide.isin([id]),:]
    points=[Point(i[0],i[1]) for i in adata2.obs[['x','y']].values]
    if mask_gdf.shape[0]==2:
        sample_ids=[0,1]
    else:
        sample_ids=[0]
    for sample in sample_ids
    ## selecting cells masked by each individual samples
        points_df=gpd.GeoDataFrame({'geometry': points})
        points_df.index=adata2.obs.index
        points_df=gpd.sjoin(points_df,mask_gdf.iloc[[sample]])
        adata2=adata2[points_df.index,:]
        
        distances = np.array([signed_distance_bv(pt, bv_gdf) for pt in points])     
        points_df['signed_distance_BV']=distances
        
df=pd.concat(df,axis=0)
df['celltype']=adata.obs.loc[df.index,'celltype']
df['subclass']=adata.obs.loc[df.index,'subclass']
df.to_csv("signed_distance_BV.tsv",sep="\t",index=True)



In [ ]:
## 06 Composition over distance bins

import matplotlib.pyplot as plt

#interface
bins=[-4000,-2000,-1500,-1000,-500,-250,0,250,500,1000,1500,2000,4000]
df_interface['distance_bin'] = pd.cut(df_interface['signed_distance_interface'], bins)
composition = df_interface.groupby(['distance_bin', 'subclass']).size().reset_index(name='count')
composition['proportion'] = composition.groupby('distance_bin')['count'].transform(lambda x: x / x.sum())
plt.figure(figsize=(10, 5))
composition_pivot = composition.pivot(index="distance_bin", columns="subclass", values="proportion").fillna(0)
composition_pivot.plot(kind="bar", stacked=True, colormap="tab20", figsize=(10, 5))
#BV
bins=[-1000,-100,0,50,100,150,200,300,500,1000]
df_bv['distance_bin'] = pd.cut(df_bv['signed_distance_bv'], bins)
composition = df_bv.groupby(['distance_bin', 'subclass']).size().reset_index(name='count')
composition['proportion'] = composition.groupby('distance_bin')['count'].transform(lambda x: x / x.sum())
plt.figure(figsize=(10, 5))
composition_pivot = composition.pivot(index="distance_bin", columns="subclass", values="proportion").fillna(0)
composition_pivot.plot(kind="bar", stacked=True, colormap="tab20", figsize=(10, 5))

In [ ]:
##07 EVT Density Analysis

from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
import geopandas as gpd
## centroids of vessels
files=[i for i in os.listdir("/path/to/geojson_dir/") if '_BV.geojson' in i]
for i in files:
    bvs=gpd.read_file(i)
    bvs=bvs.drop(bvs.shape[0]-1)
    id=i.str.replace("_BV.geojson","")
    allpoints=adata.obs.loc[(adata.obs.subclass!='EVT')&(adata.obs.slide==id)&(adata.obs.distance_to_interface.gt(-200)),['x','y']]
    random_points=np.asarray(allpoints.sample(n=10)) ## sampling cells in maternal side
    cell_coords = np.array(adata.obs.loc[(adata.obs.subclass=='EVT')&(adata.obs.slide==id),['x','y']])
    #tree = KDTree(cell_coords)
    radius = 50
    area,nevts,nevts_ctrl=[],[],[]
    for idx,p in enumerate(bvs.geometry):
        boundary_points = list(p.exterior.coords)
        buffers = [Point(pt).buffer(radius) for pt in boundary_points] ## setting annular regions near the vessel wall
        combined_buffer = unary_union(buffers)
        annular_region = combined_buffer.difference(p)
    
# Identify cells within the annular region
        cells_in_annulus = []
        for coord in cell_coords:
            point = Point(coord)
            if annular_region.contains(point):
                cells_in_annulus.append(coord)
        cells_in_annulus = np.array(cells_in_annulus)
        print(annular_region.area,cells_in_annulus.shape[0],cells_in_annulus.shape[0]/annular_region.area)
        area.append(annular_region.area)
        nevts.append(cells_in_annulus.shape[0])
    
        equivalent_radius = np.sqrt(annular_region.area/ np.pi)
        tmp=[]
        for i in random_points:
            random_center = Point(i[0], i[1])
            random_circle = random_center.buffer(equivalent_radius)
            cells_in_random_area = []
            for coord in cell_coords:
                point = Point(coord)
                if random_circle.contains(point):
                    cells_in_random_area.append(coord)
            tmp.append(len(cells_in_random_area))
        nevts_ctrl.append(np.asarray(tmp).mean())

    ## summarize the sampling
    area_df=pd.DataFrame()
    area_df['area']=area
    area_df['nevts']=nevts
    area_df['nevts_ctrl']=nevts_ctrl
    area_df['density']=area_df.nevts/area_df.area
    print(ss.ranksums(area_df.nevts,area_df.nevts_ctrl))
    area_df.to_csv(id+"EVT_density.tsv",sep="\t",index=True)